[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/data-platforms/databricks/databricks-spark-connector.ipynb)

# Databricks Spark Connector and Weaviate

This notebook shows you how to ingest data from a Spark DataFrame into Weaviate.

You'll learn how to:

1. Load data from the Databricks Marketplace

1. Ingest data into a Weaviate collection using the Weaviate Spark connector

1. Run vector search, hybrid search, and generative search over your data

## Dependencies

Check that the Weaviate client and spark connector are installed.

In [0]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.12.0
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages
Requires: authlib, grpcio, grpcio-health-checking, grpcio-tools, httpx, pydantic, validators
Required-by: langchain-weaviate


In [0]:
%sql
LIST JAR io_weaviate_spark_connector_2_12_1_3_3.jar

Results
spark://10.213.190.200:38041/jars/io_weaviate_spark_connector_2_12_1_3_3.jar


## Import Libraries

In [0]:
import weaviate
import weaviate.classes.config as wvcc
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import StringType, BooleanType
from weaviate.util import generate_uuid5
import time
import json
from pyspark.sql import Row
from py4j.protocol import Py4JJavaError

## Weaviate Cloud Setup

### Grab the secret keys

In [0]:
SCOPE="spark-connector-demo"

WEAVIATE_URL = dbutils.secrets.get(scope=SCOPE, key="WEAVIATE_URL")
WEAVIATE_API_KEY = dbutils.secrets.get(scope=SCOPE, key="WEAVIATE_API_KEY")
DATABRICKS_TOKEN = dbutils.secrets.get(scope=SCOPE, key="DATABRICKS_TOKEN")
EMBEDDINGS_ENDPOINT = dbutils.secrets.get(scope=SCOPE, key="EMBEDDINGS_ENDPOINT")
LLM_ENDPOINT = dbutils.secrets.get(scope=SCOPE, key="LLM_ENDPOINT")

### Connect to Weaviate

In [0]:
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
      headers={ "X-Databricks-Token": DATABRICKS_TOKEN}
)

print(client.is_ready())

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:585: DeprecationWarning: Passing 'msg' argument to Task.cancel() is deprecated since Python 3.11, and scheduled for removal in Python 3.14.
  should_retry = scope._deliver_cancellation(origin) or should_retry
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:585: DeprecationWarning: Passing 'msg' argument to Future.cancel() is deprecated since Python 3.11, and scheduled for removal in Python 3.14.
  should_retry = scope._deliver_cancellation(origin) or should_retry


True


In [0]:
print(f"Weaviate server version: {client.get_meta()['version']}")

Weaviate server version: 1.29.1


## Dataset

We'll ingest the products and descriptions into Weaviate

In [0]:
table_name = "bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_best_seller_products"

df = spark.table(table_name).select("title", "description").limit(10)
display(df)

title,description
YOOLIFE Initial Jute Tote Bag with Makeup Bag Beach Bag Tote with Zipper Embroidery Monogram Friend Birthday Gifts for Women,Previous page Use for Multiple Occasions Use for Multiple Occasions Use for Multiple Occasions Next page 1 Restaurant 2 Library 3 Beach Various Ways to Wear Show The Details Inside Pocket Top Zipper Strong Stitches Waterproof Surface PERFECT PERSONALIZED GIFTS Makeup Bag Initial Tote Bag with Makeup Bag
"ESOW Paper Towel Holder with Shelf Storage, Adhesive Wall Mount 2-in-1 Basket Organizer for Kitchen & Bathroom, Durable Metal Wire Design, Stainless Steel 304 Brushed Nickel Finish, 2 Packs","About this item [Premium Material & Sturdy Construction]: Made of 304 stainless steel and adopted sandblasting technology on the surface to carry out secondary protection, the paper towel holders are waterproof, moisture-proof and rust free, won’t deform when bearing great weight, looks new after long-term use. There is a removable rod to keep the paper towel spin smoothly and steady. Don't worry that the paper towel will fall down when you tear the sheet away. Hollowed bottom, drips fast and is easy to dry. [Large and Versatile Paper Towel Holders & 2 Pack]: It comes with 2 pack paper towel holders. Each shelf storage in a convenient adhesive wall mount rack with attached paper towel holder bar; Shelf can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. The towel bar is open on one end so that you can easily slide your paper towels in place. [Large and Versatile Paper Towel Holders & 2 Pack]: It comes with 2 pack paper towel holders. Each shelf storage in a convenient adhesive wall mount rack with attached paper towel holder bar; Shelf can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. The towel bar is open on one end so that you can easily slide your paper towels in place. [Multiple applications]: Shelfs can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. Smooth surfaces, no sharp edge, and all details we care are for your best to use it. They could be installed in bathroom, kitchen, bedroom, living room, dormitory, washbasin and other places, suitable for both corners and straight walls. Apartment essentials for first apartment! [100% Customer Satisfaction Guarantee]: Each accessory passed 300,000 times load bearing tests, ensuring reliable product performance and durable high quality for customer life using, is built to last. If our ESOW Paper towel holder Shelf Storage does not meet your expectations, return it for 100% refund of your purchase price."
"Solino Home Printed Linen Table Runner 14 x 108 Inch – 100% Pure Linen Blue and White Table Runner for for Indoor, Outdoor – Coastal Raindrops, Handcrafted from European Flax","About this item Coastal Raindrops Table Runner is handcrafted from 100% Pure Linen - the raindrop like pattern brings a splash of color and beautiful texture that is perfectly suited for spring, summer, fall and winter. Size - 14 x 108 Inch. Available in different sizes to fit your needs. Easy Care - Machine Washable, Low Iron as Needed, Steam Iron for a crisp look. The beauty of each pure linen piece is the natural fibers and classic aesthetic that mellows with each washing, betters with age and requires minimal care. Each batch of our linen is different and unique. Slight color and texture variations are an inherent quality of handcrafted pieces, adding to their relaxed beauty and bringing a wonderful heirloom feel to each piece. Create a monochromatic or mix n’match look to fit your style with our coordinating products from this collection. Available in sizes and styles for celebrating weddings, bridal showers, baby showers and special occasions, entertaining for Valentine’s Day, St. Patrick’s Day, Easter, Passover, Memorial

Add a UUID to each object

In [0]:

def create_uuid(title, description):
    obj = {
        "title": title,
        "description": description
    }
    return str(generate_uuid5(obj))

create_uuid_udf = udf(create_uuid, StringType())

df = df.withColumn("uuid", create_uuid_udf(df["title"], df["description"]))\
        .select("uuid", "title", "description")

display(df)

uuid,title,description
1ed9371c-4277-5afa-b481-df218e942d65,YOOLIFE Initial Jute Tote Bag with Makeup Bag Beach Bag Tote with Zipper Embroidery Monogram Friend Birthday Gifts for Women,Previous page Use for Multiple Occasions Use for Multiple Occasions Use for Multiple Occasions Next page 1 Restaurant 2 Library 3 Beach Various Ways to Wear Show The Details Inside Pocket Top Zipper Strong Stitches Waterproof Surface PERFECT PERSONALIZED GIFTS Makeup Bag Initial Tote Bag with Makeup Bag
000bb2a0-5b0f-536b-a879-e9908da3fd7e,"ESOW Paper Towel Holder with Shelf Storage, Adhesive Wall Mount 2-in-1 Basket Organizer for Kitchen & Bathroom, Durable Metal Wire Design, Stainless Steel 304 Brushed Nickel Finish, 2 Packs","About this item [Premium Material & Sturdy Construction]: Made of 304 stainless steel and adopted sandblasting technology on the surface to carry out secondary protection, the paper towel holders are waterproof, moisture-proof and rust free, won’t deform when bearing great weight, looks new after long-term use. There is a removable rod to keep the paper towel spin smoothly and steady. Don't worry that the paper towel will fall down when you tear the sheet away. Hollowed bottom, drips fast and is easy to dry. [Large and Versatile Paper Towel Holders & 2 Pack]: It comes with 2 pack paper towel holders. Each shelf storage in a convenient adhesive wall mount rack with attached paper towel holder bar; Shelf can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. The towel bar is open on one end so that you can easily slide your paper towels in place. [Large and Versatile Paper Towel Holders & 2 Pack]: It comes with 2 pack paper towel holders. Each shelf storage in a convenient adhesive wall mount rack with attached paper towel holder bar; Shelf can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. The towel bar is open on one end so that you can easily slide your paper towels in place. [Multiple applications]: Shelfs can make you saving a large number of space which can storage shelf for spices, cleaning supplies, soaps, candles, lotions and others. Smooth surfaces, no sharp edge, and all details we care are for your best to use it. They could be installed in bathroom, kitchen, bedroom, living room, dormitory, washbasin and other places, suitable for both corners and straight walls. Apartment essentials for first apartment! [100% Customer Satisfaction Guarantee]: Each accessory passed 300,000 times load bearing tests, ensuring reliable product performance and durable high quality for customer life using, is built to last. If our ESOW Paper towel holder Shelf Storage does not meet your expectations, return it for 100% refund of your purchase price."
1734c0c9-d29f-5943-9be4-6ddf622e2d35,"Solino Home Printed Linen Table Runner 14 x 108 Inch – 100% Pure Linen Blue and White Table Runner for for Indoor, Outdoor – Coastal Raindrops, Handcrafted from European Flax","About this item Coastal Raindrops Table Runner is handcrafted from 100% Pure Linen - the raindrop like pattern brings a splash of color and beautiful texture that is perfectly suited for spring, summer, fall and winter. Size - 14 x 108 Inch. Available in different sizes to fit your needs. Easy Care - Machine Washable, Low Iron as Needed, Steam Iron for a crisp look. The beauty of each pure linen piece is the natural fibers and classic aesthetic that mellows with each washing, betters with age and requires minimal care. Each batch of our linen is different and unique. Slight color and texture variations are an inherent quality of handcrafted pieces, adding to their relaxed beauty and bringing a wonderful heirloom feel to each piece. Create a monochromatic or mix n’match look to fit your style with our coordinating products from this collection. Available in sizes and styles for celebrating weddings, bridal showers,

## Create the Weaviate Collection

In [0]:
if client.collections.exists("AmazonProducts"):
    client.collections.delete("AmazonProducts")

collection = client.collections.create(
    name="AmazonProducts",
    vectorizer_config=[
        wvcc.Configure.NamedVectors.text2vec_databricks(
            name="default", 
            endpoint=EMBEDDINGS_ENDPOINT
        )
    ],
    generative_config=wvcc.Configure.Generative.databricks(
        endpoint=LLM_ENDPOINT, 
        max_tokens=8092
    ),
    properties=[
        wvcc.Property(
            name="title",
            data_type=wvcc.DataType.TEXT
        ),
        wvcc.Property(
            name="description",
            data_type=wvcc.DataType.TEXT
        )
    ]
)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:585: DeprecationWarning: Passing 'msg' argument to Task.cancel() is deprecated since Python 3.11, and scheduled for removal in Python 3.14.
  should_retry = scope._deliver_cancellation(origin) or should_retry
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/anyio/_backends/_asyncio.py:585: DeprecationWarning: Passing 'msg' argument to Future.cancel() is deprecated since Python 3.11, and scheduled for removal in Python 3.14.
  should_retry = scope._deliver_cancellation(origin) or should_retry
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages/weaviate/collections/classes/config.py:2082: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in 

## Ingest the data into Weaviate 

In [0]:
df.write.format("io.weaviate.spark.Weaviate") \
    .option("batchSize", 200) \
    .option("scheme", "https") \
    .option("host", WEAVIATE_URL.replace("https://","")) \
    .option("apiKey", WEAVIATE_API_KEY) \
    .option("header:X-Databricks-Token", DATABRICKS_TOKEN) \
    .option("className", "AmazonProducts") \
    .option("id", "uuid") \
    .mode("append") \
    .save()


Count the number of objects in the `AmazonProducts` collection

In [0]:
products = client.collections.get("AmazonProducts")

total_count_in_weaviate = products.aggregate.over_all(total_count=True).total_count
total_count_in_weaviate

10

## Search

In [0]:
# Get the `AmazonProducts` collection 
products = client.collections.get("AmazonProducts")

## Vector Search

Vector search returns the objects with most similar vectors to that of the query.

In [0]:
response = products.query.near_text(
    query="home essentials",
    limit=2
)

for item in response.objects:
    print(json.dumps(item.properties, indent=2), "\n")

{
  "title": "Bare Home 100% Organic Cotton Queen Sheet Set - Crisp Percale Weave - Lightweight & Breathable - Bedding Sheets & Pillowcases (Queen, Dusty Pink)",
  "description": "About this item SET THE STANDARD: Our Queen Percale Sheet Set is crisp to the touch and beautiful to look at. Our 4-piece queen sheet set is made to impress from first rest and gets softer with each wash. SET INCLUDES: 1 Queen Fitted Sheet (60\" x 80\" x 15\" deep pocket), 1 Queen Flat Sheet (90\" x 102\"), 2 Standard Pillowcases (20\" x 27\"). Fits mattresses up to 15\u201d deep. SLEEP IN LUXURY: At the end of the day, you deserve a crisp feel of hotel luxury. So, go ahead. Slip into Bare Home's lightweight and crisp percale sheets. Made from 100% organic cotton and organic dyes. PURELY ORGANIC: Bare Home organic products meet the highest levels of environmental and social standards from the field to your home. But we don\u2019t stop there. Many chemicals used in fabric production-such as aluminum and formal

### Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search.

Alpha is an optional parameter. The default is set to `0.75`.

In [0]:
response = products.query.hybrid(
    query="stay dry in the rain",
    query_properties=["title"],
    alpha=0.7,
    limit=2
)

for item in response.objects:
    print(json.dumps(item.properties, indent=2), "\n")


{
  "title": "Columbia Women's Top Pine Insulated Rain Jacket",
  "description": "Mountain weather can be fickle\u2014best to be prepared with a jacket that\u2019s as waterproof as it is warm. Crafted of a durable and breathable Omni-TECH\u2122 shell, with a storm hood and drop tail for extra coverage, this insulated layer is exactly that."
} 

{
  "title": "Solino Home Printed Linen Table Runner 14 x 108 Inch \u2013 100% Pure Linen Blue and White Table Runner for for Indoor, Outdoor \u2013 Coastal Raindrops, Handcrafted from European Flax",
  "description": "About this item Coastal Raindrops Table Runner is handcrafted from 100% Pure Linen - the raindrop like pattern brings a splash of color and beautiful texture that is perfectly suited for spring, summer, fall and winter. Size - 14 x 108 Inch. Available in different sizes to fit your needs. Easy Care - Machine Washable, Low Iron as Needed, Steam Iron for a crisp look. The beauty of each pure linen piece is the natural fibers and cla

### Generative Search (RAG)

In [0]:
generate_prompt = "Explain why each product is needed in my home."

response = products.generate.near_text(
    query="home essentials",
    return_properties=["title", "description"],
    grouped_task=generate_prompt,
    limit=2
)

print(response.generated)

Here's why each product is needed in your home:

1. **Bare Home 100% Organic Cotton Queen Sheet Set**: This product is needed in your home for a comfortable and healthy sleep. The sheet set is made of 100% organic cotton, which is breathable, soft, and gentle on your skin. It's also free of toxic chemicals, ensuring a safe and healthy sleep environment. Additionally, the sheet set is machine washable and becomes softer with every wash, making it a practical and durable choice for your bedroom.

2. **ESOW Paper Towel Holder with Shelf Storage**: This product is needed in your home for convenience and organization. The paper towel holder with shelf storage provides a compact and space-saving solution for storing paper towels, spices, cleaning supplies, and other essentials in your kitchen, bathroom, or other rooms. The adhesive wall mount design makes it easy to install, and the stainless steel construction ensures durability and rust-resistance. The shelf storage also helps keep your co